# Spacy/Medspacy IAA

## Resources

Prodigy forum answer about IAA for spans https://support.prodi.gy/t/proper-way-to-calculate-inter-annotator-agreement-for-spans-ner/5760

Spacy scorer object https://spacy.io/api/scorer

## End Goal

### Functionality

Provide a collection of methods to evaluate IAA between _n_ arbitrary spacy `doc` objects. Provide methods that aid in error analysis such as providing lists of differences.

Priorities:
* Pairwise F1
    * configurable strict/loose matching
    * configurable inclusion of labels/attributes (calculate just span vs span+class agreement)

* Imported python files
    * reasonable docstrings on methods/classes
    
* Unit tests
    * add CI to repo for automated testing later

Extra features:
* List of differences between docs
* 

Expected challenges
* Spacy scorer functions are useful, but _only_ do strict span matching
* Fewer resources (obviously?) available for comparisons between 3+ docs


# Testing Section

In [1]:
import spacy

In [ ]:
#!python -m spacy download en_core_web_md

In [2]:
nlp = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_core_web_md")

In [3]:
doc = nlp("this is a test document made in utah or mississippi, or salt lake city.")

In [4]:
doc.ents

(utah, mississippi)

In [5]:
doc2 = nlp2("this is a test document made in utah or mississippi, or salt lake city.")

In [6]:
doc2.ents

(utah, mississippi, lake city)

In [26]:
from spacy.tokens import Span
spand = list()
spand += [Span(doc, 2, 4, label="PERSON"),Span(doc,7,8,label="GPE"),Span(doc,9,10,label="PERSON"),Span(doc,13,14,label="GPE"),Span(doc,14,15,label="GPE")]

# Add the span to the doc's entities
doc.ents = spand

# Print entities' text and labels
print([(ent.text, ent.label_) for ent in doc.ents])

[('a test', 'PERSON'), ('utah', 'GPE'), ('mississippi', 'PERSON'), ('lake', 'GPE'), ('city', 'GPE')]


In [66]:
#Run below cells before this
tp,fp,fn = agreement(doc,doc2,1,1)
print(tp,fp,fn)
print(pairwise_f1(tp,fp,fn))

2 1 2
0.5714285714285714


# Code

In [61]:
from quicksectx import IntervalNode, IntervalTree, Interval #note that you need the quicksectx library

#In order to make the code a little more adaptable for situations of multiple overlapping entities, as well as for 
#transparency and testing the code, I wrote the overlaps code to output a mapping of which entities are being matched. 
#Then agreement can parse this output for how many valid overlaps exist.

#This makes the code a little more complicated to understand, but I think it makes everything more transparent and adaptable.

def overlaps(doc1_ents, doc2_ents,labels):
    '''Calculates overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()
    
    tree = IntervalTree()
    for index2,ent2 in enumerate(doc2_ents):
        tree.add(ent2.start_char,ent2.end_char,index2)
    
    for index1,ent1 in enumerate(doc1_ents):
        matches = tree.search(ent1.start_char,ent1.end_char)
        for match in matches:
            index2 = match.data #match.data is the index of doc2_ents
            if ((labels == 0) | (doc2_ents[index2].label_ == ent1.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches

In [16]:
### This is the old, less efficient code. The newer code uses a tree search instead of the nested for-loop.

def old_overlaps(doc1_ents, doc2_ents,labels):
    '''Old code for calculating overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    for index1,ent1 in enumerate(doc1_ents):
        for index2,ent2 in enumerate(doc2_ents):
            if (ent1.end_char >= ent2.start_char) & (ent1.start_char <= ent2.end_char) & ((labels==0) | (ent1.label_ == ent2.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches
    

In [62]:
def exact_match(doc1_ents, doc2_ents, labels):
    '''calculate whether two ents have exact overlap
    returns bool
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    doc1_ent_dict = dict()
    doc2_ent_dict = dict()
    
    for index1,ent1 in enumerate(doc1_ents):
        if labels == 1: #If checking for labels, then include this in the tuple's to-be-compared elements
            doc1_ent_dict[(ent1.start_char,ent1.end_char,ent1.label_)] = index1
        else:
            doc1_ent_dict[(ent1.start_char,ent1.end_char)] = index1
            
    for index2,ent2 in enumerate(doc2_ents):
        if labels == 1:    
            doc2_ent_dict[(ent2.start_char,ent2.end_char,ent2.label_)] = index2
        else:
            doc2_ent_dict[(ent2.start_char,ent2.end_char)] = index2
        
    doc1_ent_set = set(doc1_ent_dict.keys())
    doc2_ent_set = set(doc2_ent_dict.keys())
    
    matched_ents = doc1_ent_set.intersection(doc2_ent_set)
    
    for match in matched_ents:
        index1 = doc1_ent_dict[match]
        index2 = doc2_ent_dict[match]
        doc1_matches[index1] = [index2]
        doc2_matches[index2] = [index1]
        
    return doc1_matches, doc2_matches
    

In [63]:
def agreement(doc1, doc2, loose=1, labels=1):
    '''Calculates confusion matrix for agreement between two documents.
    
       returns true positive, false positive, and false negative
    '''
    tp = 0 #True Positives
    fp = 0 #False Positives
    fn = 0 #False Negatives
    
    doc1_ents = doc1.ents
    doc2_ents = doc2.ents
    
    if loose:
        doc1_matches, doc2_matches = overlaps(doc1_ents, doc2_ents, labels)
    else:
        doc1_matches, doc2_matches = exact_match(doc1_ents, doc2_ents, labels)
    
    doc1_match_num = len(doc1_matches.keys())
    doc2_match_num = len(doc2_matches.keys())
    
    duplicate_matches = 0
    for value in doc2_matches.values():
        duplicate_matches += len(value) - 1
    
    tp = doc1_match_num - duplicate_matches #How many entity indices from doc1 matched, minus duplicated matches
    fp = len(doc2_ents) - doc2_match_num #How many entities from doc2 that didn't match
    fn = len(doc1_ents) - doc1_match_num #How many entities from doc1 that didn't match
    
    return (tp, fp, fn)


In [64]:
def pairwise_f1(tp,fp,fn):
    '''calculate f1 given true positive, false positive, and false negative values'''
    
    return (2*tp)/float(2*tp+fp+fn)

In [65]:
def corpus_agreement(docs1, docs2, loose=1, labels=1):
    '''calculate f1 over an entire corpus of documents'''
    corpus_tp, corpus_fp, corpus_fn = (0,0,0)
    
    for i, doc1 in enumerate(docs1):
        tp,fp,fn = agreement(doc1, docs2[i],loose)
        corpus_tp += tp
        corpus_fp += fp
        corpus_fn += fn
    
    return pairwise_f1(tp,fp,fn)